In [6]:
pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/a8/e9/3e4879974a7c4dcaca2a746dde3df08d0ae8f14c74b03591616ce5f0a8b1/scikit_learn-1.4.0-1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 1.3 MB/s eta 0:00:09
    --------------------------------------- 0.1/10.6 MB 1.7 MB/s eta 0:00:07
   - -------------------------------------- 0.3/10.6 MB 2.1 MB/s eta 0:00:05
   - -------------------------------------- 0.5/10.6 MB 2


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import warnings
import datetime
import time
import threading
import concurrent.futures
import multiprocessing

In [2]:
from datetime import timedelta
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint
#from sklearn.cluster import DBSCAN
#from sklearn.cluster import KMeans
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from itertools import combinations
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression
from multiprocessing import Pool, Process, Queue
#from pmdarima import auto_arima
#from statsmodels.tsa.arima.model import ARIMA
#from googletrans import Translator

In [3]:
def get_sp500_tickers():
    # Ссылка на страницу Wikipedia с перечнем компаний S&P 500
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

    # Скачивание таблицы с помощью pandas
    sp500_table = pd.read_html(url, header=0)[0]
    tickers = sp500_table['Symbol'].tolist()

    # Обработка символов, которые могут вызвать проблемы (например, замена '.' на '-')
    tickers = [ticker.replace('.', '-') for ticker in tickers]

    return tickers

# Получение списка тикеров
tickers = get_sp500_tickers()

start_date = "2022-01-02"
end_date = "2024-01-22"
data = yf.download(tickers, start=start_date, end=end_date)['Close']

data.head()

[*********************100%%**********************]  503 of 503 completed


Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,156.479996,18.750000,182.009995,135.419998,172.679993,139.039993,44.549999,407.209991,564.369995,177.149994,...,87.769997,67.940002,63.540001,56.889999,116.889999,136.529999,125.368935,583.900024,64.239998,234.029999
2022-01-04,151.190002,19.020000,179.699997,135.160004,170.800003,135.770004,45.130001,404.299988,554.000000,175.550003,...,87.489998,68.370003,65.930000,57.180000,118.129997,137.259995,126.640778,587.599976,66.070000,225.119995
2022-01-05,148.600006,18.680000,174.919998,135.869995,162.250000,135.160004,44.599998,397.179993,514.429993,172.839996,...,84.250000,69.040001,66.750000,56.259998,116.519997,135.520004,126.223305,558.179993,66.220001,216.559998
2022-01-06,149.119995,18.570000,172.000000,135.229996,159.750000,135.139999,44.860001,378.000000,514.119995,173.389999,...,84.639999,68.500000,68.320000,56.220001,116.360001,136.919998,125.174759,555.159973,68.940002,217.419998
2022-01-07,145.149994,19.280001,172.169998,134.880005,166.050003,135.559998,45.070000,370.750000,510.700012,168.839996,...,84.550003,69.099998,68.879997,56.400002,115.430000,135.350006,124.417473,530.859985,70.089996,211.089996


In [4]:
# Удаляем столбцы, где более 10% данных отсутствуют
threshold = 0.1  # 10% порог
data = data.dropna(thresh=int((1 - threshold) * len(data)), axis=1)

# Заполнение оставшихся пропусков, например, методом ffill (forward fill)
data = data.ffill()

# Удаляем строки, которые все еще содержат NaN (если они есть)
data = data.dropna()

In [5]:
pairs = list(combinations(data.columns, 2))
pairs[:10]

[('A', 'AAL'),
 ('A', 'AAPL'),
 ('A', 'ABBV'),
 ('A', 'ABNB'),
 ('A', 'ABT'),
 ('A', 'ACGL'),
 ('A', 'ACN'),
 ('A', 'ADBE'),
 ('A', 'ADI'),
 ('A', 'ADM')]

In [6]:
def process_pair(pair):
    # Тестируемые тикеры акций
    name_1, name_2 = pair

    ticker1 = data[name_1]
    ticker2 = data[name_2]

    # Объединение данных по дате
    z_test = pd.merge(ticker1, ticker2, left_index=True, right_index=True, how='inner')
    z_test.columns = ['JBHT_Close', 'LOW_Close']

    # Расчет логарифмической разности
    z_test['Spread'] = np.log(z_test['LOW_Close']) - np.log(z_test['JBHT_Close'])

    # Перевод дат в цифровой формат
    z_test['DateNumeric'] = z_test.index.map(pd.Timestamp.toordinal)

    # Создание индексов в качестве признаков
    X = z_test['DateNumeric'].values.reshape(-1, 1)

    # Начальные параметры
    initial_capital = 100000    # Стартовый капитал
    max_cash = initial_capital    # Максимально достигнутая сумма
    cash = initial_capital    # Баланс кошелька
    low_shares = 0    # Акции low
    jbht_shares = 0    # Акции jbht
    trade_count = 0   # Счетчик сделок
    window = 28  # Окно для скользящего среднего и стандартного отклонения
    roll_window = 23  # Окно для линейной регрессии
    red_flag = True # Индикатор стоп-лосса

    for index in range(window, len(z_test)):

        current_date = z_test.index[index]
        jbht_price = z_test.at[current_date, 'JBHT_Close']
        low_price = z_test.at[current_date, 'LOW_Close']
        # Расчет скользящего среднего и стандартного отклонения для спреда
        mean_spread = z_test['Spread'][index-window:index].mean()
        std_spread = z_test['Spread'][index-window:index].std()
        z_score = (z_test.at[current_date, 'Spread'] - mean_spread) / std_spread

        # Расчет отклонения от линии регрессии котировок
        numeric_date = current_date.toordinal()
        # Модель для JBHT
        model_jbht = LinearRegression()
        model_jbht.fit(X[index-roll_window:index], z_test['JBHT_Close'][index-roll_window:index])
        predicted_jbht = model_jbht.predict([[numeric_date]])

        # Модель для LOW
        model_low = LinearRegression()
        model_low.fit(X[index-roll_window:index], z_test['LOW_Close'][index-roll_window:index])
        predicted_low = model_low.predict([[numeric_date]])

        # Расчет отклонения от линии регрессии
        deviation_jbht = jbht_price - predicted_jbht
        deviation_low = low_price - predicted_low

        # Устанавливаем пороги на основе волатильности
        entry = 42 * std_spread
        exit = 1 * std_spread

        # Торговые сигналы и сделки
        # Покупка спреда
        if z_score < -entry:
            # Длинная позиция по дорогой, короткая по дешевой
            if deviation_jbht >= deviation_low:
                if low_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 1
                elif jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
            # Короткая позиция по LOW, длинная по JBHT
            else: #elif red_flag == False and z_test.at[current_date, 'LOW_Close'] > z_test.at[current_date, 'JBHT_Close']:
                if low_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash += -low_shares * low_price
                    trade_count += 1
                elif low_shares < 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1

        # Покупка спреда
        elif z_score > entry:
            # Короткая позиция по LOW, длинная по JBHT
            if deviation_jbht < deviation_low:
                if low_shares == 0:
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    cash -= jbht_shares * jbht_price
                    cash -= low_shares * low_price
                    trade_count += 1
                elif low_shares < 0:
                    delta = (-cash / low_price - low_shares) * low_price
                    jbht_shares -= delta / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
                elif jbht_shares < 0:
                    cash += low_shares * low_price
                    cash += jbht_shares * jbht_price
                    low_shares = 0
                    jbht_shares = 0
                    jbht_shares = cash / jbht_price
                    low_shares = -(cash / low_price)
                    trade_count += 1
            # Длинная позиция по LOW, короткая по JBHT
            else: #elif red_flag == False and z_test.at[current_date, 'LOW_Close'] > z_test.at[current_date, 'JBHT_Close']:
                if low_shares == 0:
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    cash -= low_shares * low_price
                    cash += -jbht_shares * jbht_price
                    trade_count += 1
                elif jbht_shares < 0:
                    delta = (-cash / jbht_price - jbht_shares) * jbht_price
                    low_shares -= delta / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1
                elif low_shares < 0:
                    cash += jbht_shares * jbht_price
                    cash += low_shares * low_price
                    low_shares = 0
                    jbht_shares = 0
                    low_shares = cash / low_price
                    jbht_shares = -(cash / jbht_price)
                    trade_count += 1

        # Выход из позиции
        elif -exit < z_score < exit:
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1

        # Установка stop-loss
        # Расчет текущей стоимости позиций         
        value_low = low_shares * low_price
        value_jbht = jbht_shares * jbht_price
        total_value = cash + value_low + value_jbht

        if max_cash < cash:
            max_cash = cash
        stop_loss = 0.15 * max_cash

        # Расчет текущей позиции
        current_loss = max_cash - total_value

        # Проверка условий стоп-лосса
        if current_loss > stop_loss:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1
            max_cash = 0
            #red_flag = not red_flag

        # Установка take-profit
        stop_profit = 0.17 * max_cash

        # Расчет текущего профита
        current_profit = total_value - max_cash

        if current_profit > stop_profit:
            # Закрытие всех позиций
            cash += low_shares * low_price
            cash += jbht_shares * jbht_price
            low_shares = 0
            jbht_shares = 0
            trade_count += 1

        # Если есть шорт позиции, списываем комиссию за день
        if jbht_shares < 0:
            daily_fee = -jbht_shares * jbht_price * 0.13 / 365
            cash -= daily_fee
        elif low_shares < 0:
            daily_fee = -low_shares * low_price * 0.13 / 365
            cash -= daily_fee
                
            final_result = cash
            final_trade_count = trade_count      
    return {'pair': pair, 'final_result': final_result, 'trade_count': trade_count}

In [6]:
test_pair = pairs[:100]

In [13]:
# Тест быстродействия
result_test = []
start_time = time.time()
for pair in test_pair:
    result_test.append(process_pair(pair))
end_time = time.time()
print(end_time - start_time)

32.758411169052124


In [ ]:
# Использование ThreadPoolExecutor для многопоточной обработки
result_test = []
start_time = time.time()
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    # Запуск функции process_pair на каждой паре в списке pairs
    futures = [executor.submit(process_pair, pair) for pair in pairs[:10]]

    for future in concurrent.futures.as_completed(futures):
        result_test.append(future.result())
end_time = time.time()
print(end_time - start_time)

In [7]:
def test_function(pair):
    name_1, name_2 = pair
    name_tick1 = str(name_1)
    name_tick2 = str(name_2)
    return (name_tick1, name_tick2)

In [ ]:
result_test = []
start_time = time.time()
# Использование ThreadPoolExecutor для многопоточной обработки
with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
    # Запуск функции process_pair на каждой паре в списке pairs
    results = executor.map(test_function, pairs[:10])

    for i in results:
        result_test.append(i)
end_time = time.time()
print(end_time - start_time)

In [20]:
test_function(test_pair[1])

('A', 'AAPL')

In [8]:
def chunkify(lst, n):
    """ Разделение списка на n подсписков """
    return [lst[i::n] for i in range(n)]

def worker(data, queue):
    print(f"Обработка данных: {data}")  # Логирование
    for item in data:
        result = test_function(item)
        queue.put(result)
    print(f"Обработка завершена для данных: {data}")  # Логирование

if __name__ == '__main__':
    start_time = time.time()

    num_processes = 4  # Количество процессов
    
    # Предположим, что test_pair - это список кортежей
    data_chunks = chunkify(test_pair, num_processes)
    queue = multiprocessing.Queue()

    processes = [multiprocessing.Process(target=worker, args=(chunk, queue)) for chunk in data_chunks]

    for p in processes:
        p.start()

    for p in processes:
        p.join()

    results = []
    while not queue.empty():
        results.append(queue.get())

    end_time = time.time()
    print(end_time - start_time)
    print(results)  # Вывод результатов

0.06100106239318848
[]


In [17]:
def test_function(pair):
    name_1, name_2 = pair
    name_tick1 = str(name_1)
    name_tick2 = str(name_2)
    return (name_tick1, name_tick2)

def chunkify(lst, n):
    """ Разделение списка на n подсписков """
    return [lst[i::n] for i in range(n)]

def worker(data, queue):
    print(f"Запуск worker с данными: {data}")
    for item in data:
        result = test_function(item)
        queue.put(result)
    print(f"Worker завершил работу с данными: {data}")

if __name__ == '__main__':
    start_time = time.time()

    num_processes = 4

    data_chunks = chunkify(test_pair, num_processes)
    queue = multiprocessing.Queue()

    processes = [multiprocessing.Process(target=worker, args=(chunk, queue)) for chunk in data_chunks]

    for p in processes:
        p.start()

    for p in processes:
        p.join()

    results = []
    while not queue.empty():
        results.append(queue.get())

    end_time = time.time()
    print(f"Время выполнения: {end_time - start_time}")
    print(f"Результаты: {results}")

In [ ]:
queue = multiprocessing.Queue()
def worker(data, queue):
    print(f"Запуск worker с данными: {data}")
    for item in data:
        result = 'test_result'  # Заменим на простое тестовое значение
        queue.put(result)
    print(f"Worker завершил работу с данными: {data}")
worker(test_pair, queue)

Запуск worker с данными: [('A', 'AAL'), ('A', 'AAPL'), ('A', 'ABBV'), ('A', 'ABNB'), ('A', 'ABT'), ('A', 'ACGL'), ('A', 'ACN'), ('A', 'ADBE'), ('A', 'ADI'), ('A', 'ADM'), ('A', 'ADP'), ('A', 'ADSK'), ('A', 'AEE'), ('A', 'AEP'), ('A', 'AES'), ('A', 'AFL'), ('A', 'AIG'), ('A', 'AIZ'), ('A', 'AJG'), ('A', 'AKAM'), ('A', 'ALB'), ('A', 'ALGN'), ('A', 'ALL'), ('A', 'ALLE'), ('A', 'AMAT'), ('A', 'AMCR'), ('A', 'AMD'), ('A', 'AME'), ('A', 'AMGN'), ('A', 'AMP'), ('A', 'AMT'), ('A', 'AMZN'), ('A', 'ANET'), ('A', 'ANSS'), ('A', 'AON'), ('A', 'AOS'), ('A', 'APA'), ('A', 'APD'), ('A', 'APH'), ('A', 'APTV'), ('A', 'ARE'), ('A', 'ATO'), ('A', 'AVB'), ('A', 'AVGO'), ('A', 'AVY'), ('A', 'AWK'), ('A', 'AXON'), ('A', 'AXP'), ('A', 'AZO'), ('A', 'BA'), ('A', 'BAC'), ('A', 'BALL'), ('A', 'BAX'), ('A', 'BBWI'), ('A', 'BBY'), ('A', 'BDX'), ('A', 'BEN'), ('A', 'BF-B'), ('A', 'BG'), ('A', 'BIIB'), ('A', 'BIO'), ('A', 'BK'), ('A', 'BKNG'), ('A', 'BKR'), ('A', 'BLDR'), ('A', 'BLK'), ('A', 'BMY'), ('A', 'BR'), ('

In [11]:
results = []
while not queue.empty():
    results.append(queue.get())

In [16]:
def print_word(word):
    print('hello,', word)

if __name__ == '__main__':
    p1 = Process(target=print_word, args=('bob',), daemon=True)
    p2 = Process(target=print_word, args=('alice',), daemon=True)
    p1.start()
    p2.start()
    p1.join()
    p2.join()

In [16]:
with Pool() as pool:
        results = [pool.apply_async(process_data, (chunk,)) for chunk in data_chunks]
        output = [r.get() for r in results]

<Process name='Process-12' pid=19920 parent=4832 stopped exitcode=1>

In [126]:
df_2 = pd.DataFrame(list_results_2)
df_2 = df_2.sort_values(by='final_result', ascending=False)
df_2.head()

,pair,final_result,trade_count
43164,"(CI, MRO)",584396.118629,131
109025,"(MTB, ON)",569490.402345,128
93850,"(ITW, WBD)",557556.228860,116
11079,"(ALGN, PARA)",538023.244944,91
24859,"(BALL, MAA)",527943.277450,174


In [127]:
df_2[:50]

,pair,final_result,trade_count
43164,"(CI, MRO)",584396.118629,131
109025,"(MTB, ON)",569490.402345,128
93850,"(ITW, WBD)",557556.228860,116
11079,"(ALGN, PARA)",538023.244944,91
24859,"(BALL, MAA)",527943.277450,174
96288,"(K, WBD)",524544.540559,99
24902,"(BALL, NOW)",520961.184255,136
104067,"(MCK, MRO)",505756.709927,122
26027,"(BBY, CZR)",463585.846707,101
123343,"(TSN, VFC)",459849.418870,109


In [8]:
list_results = []
# Топорный тест всех пар на стратегии
for pair in pairs:
    list_results.append(process_pair(pair))
# Создание DataFrame из списка словарей
df_3 = pd.DataFrame(list_results)
df_3 = df_3.sort_values(by='final_result', ascending=False)
df_3.head()

,pair,final_result,trade_count
105914,"(MKTX, PXD)",639754.156523,91
10891,"(ALGN, ESS)",634554.861548,114
19333,"(APTV, FSLR)",613870.676615,73
62436,"(DVA, MPWR)",599487.934569,63
53180,"(CTLT, FE)",565409.504990,111


In [10]:
df_3.to_csv('SP500 pairs - 40-1std-28d.csv')

In [11]:
df_3k = df_3[:30000].copy()

In [12]:
df_3k['ADF'] = 0
df_3k['ADF-stat'] = 0.0
df_3k['ADF-p'] = 0.0
df_3k['CADF-score'] = 0.0
df_3k['CADF-p'] = 0.0
df_3k['Spearman'] = 0.0
df_3k['spear-p'] = 0.0

df_3k.head()

,pair,final_result,trade_count,ADF,ADF-stat,ADF-p,CADF-score,CADF-p,Spearman,spear-p
105914,"(MKTX, PXD)",639754.156523,91,0,0.0,0.0,0.0,0.0,0.0,0.0
10891,"(ALGN, ESS)",634554.861548,114,0,0.0,0.0,0.0,0.0,0.0,0.0
19333,"(APTV, FSLR)",613870.676615,73,0,0.0,0.0,0.0,0.0,0.0,0.0
62436,"(DVA, MPWR)",599487.934569,63,0,0.0,0.0,0.0,0.0,0.0,0.0
53180,"(CTLT, FE)",565409.504990,111,0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Тесты пар
for i, row in df_3k.iterrows():
    tik1, tik2 = row['pair']
    if tik1 in data.columns and tik2 in data.columns:
        stock1 = data[tik1]
        stock2 = data[tik2]

        # Рассчитываем коинтеграцию с помощью CADF теста
        score, p_value, _ = ts.coint(stock1, stock2)
        # ADF тест
        spread = stock1 - stock2
        adf_result = ts.adfuller(spread)
        # Корреляция Спирмена
        corr, spear_p = spearmanr(stock1, stock2)
        # Сохраняем результаты
        df_3k.at[i, 'CADF-p'] = p_value
        df_3k.at[i, 'CADF-score'] = score
        df_3k.at[i, 'ADF-p'] = adf_result[1]
        df_3k.at[i, 'ADF-stat'] = adf_result[0]
        if adf_result[0] < adf_result[4]['1%']:
            df_3k.at[i, 'ADF'] = 1
        df_3k.at[i, 'Spearman'] = corr
        df_3k.at[i, 'spear-p'] = spear_p

df_3k = df_3k.sort_values(by='Spearman')
df_3k[-20:]

,pair,final_result,trade_count,ADF,ADF-stat,ADF-p,CADF-score,CADF-p,Spearman,spear-p
116163,"(PH, TDG)",111575.585432,261,0,-0.505136,0.891010,-4.163526,0.004140,0.957710,3.362741e-273
35771,"(CAH, ORLY)",134475.904640,212,0,-0.938664,0.774973,-2.525993,0.267727,0.958002,6.156534e-274
92646,"(IR, LEN)",158190.187789,171,0,-0.469756,0.897784,-4.203570,0.003603,0.958064,4.283287e-274
99418,"(LEN, PH)",143854.990831,184,0,-0.555552,0.880713,-3.127296,0.082951,0.959518,7.471051e-278
13733,"(AME, ETN)",110149.008274,280,0,0.236694,0.974246,-2.125358,0.463121,0.960330,5.162139e-280
35539,"(CAH, CTAS)",158036.298730,215,0,-0.493943,0.893193,-3.452117,0.036883,0.960413,3.088364e-280
84456,"(HBAN, RF)",136407.162913,291,0,-2.694978,0.074911,-3.201697,0.069645,0.961516,2.992498e-283
6873,"(AEE, WEC)",116386.853713,350,0,-2.810101,0.056845,-2.637612,0.222815,0.963177,5.840341e-288
114967,"(PCAR, TT)",126204.172186,249,0,0.046302,0.962198,-3.768623,0.014918,0.963735,1.374162e-289
50373,"(CPRT, TDG)",161214.563565,223,0,0.280378,0.976426,-2.858305,0.147837,0.964138,8.796398e-291


In [22]:
df_3k = df_3k.sort_values(by='CADF-score')
df_3k[:50]

,pair,final_result,trade_count,ADF,ADF-stat,ADF-p,CADF-score,CADF-p,Spearman,spear-p
118381,"(PYPL, UPS)",196338.641338,131,1,-5.221611,7.941489e-06,-6.255725,4.468363e-07,0.773931,1.813209e-101
64207,"(ECL, NOW)",250230.069497,168,0,-0.760227,8.305270e-01,-6.204939,5.791002e-07,0.874112,3.101189e-159
118326,"(PYPL, SBAC)",117431.415760,120,0,-1.591686,4.877688e-01,-5.927188,2.327116e-06,0.833518,3.436970e-131
64192,"(ECL, MSFT)",138304.356460,213,0,-0.947931,7.717788e-01,-5.889202,2.804222e-06,0.875691,1.618952e-160
75253,"(FFIV, UNP)",291918.526612,212,1,-5.891417,2.918418e-07,-5.885032,2.862059e-06,0.898260,5.052072e-181
118352,"(PYPL, TECH)",143782.568882,96,1,-5.888546,2.961735e-07,-5.693346,7.225455e-06,0.700399,2.151600e-75
118420,"(PYPL, ZION)",118764.792557,108,1,-5.858816,3.448990e-07,-5.597966,1.134801e-05,0.831255,7.444558e-130
64169,"(ECL, MCO)",163183.198816,255,0,-1.959484,3.046157e-01,-5.431109,2.460793e-05,0.893887,1.077922e-176
118400,"(PYPL, WEC)",215043.576813,105,1,-5.563713,1.521396e-06,-5.380259,3.102671e-05,0.707282,1.735250e-77
64186,"(ECL, MPWR)",213170.468367,95,0,-2.378488,1.479002e-01,-5.375166,3.175201e-05,0.709987,2.512560e-78


In [25]:
st = sum(df_3k['final_result']/len(df_3k))
st

NameError: name 'df_3k' is not defined